In [ ]:
RFTrackPath = '/home/pkruyt/cernbox/rf-track-2.0'
import sys
import numpy as np

sys.path.append(RFTrackPath)

import RF_Track as RFT


In [ ]:

Ions_A = 207.2; #% atomic number
Ions_Q = 54;# % ion charge
Ions_mass = Ions_A * RFT.protonmass; #% MeV/c^2
#print('Ion mass:',Ions_mass)
Ions_K = 4.2 * Ions_A; #% MeV, initial kinetic energy
Ions_E = Ions_mass + Ions_K;# % MeV
Ions_P = np.sqrt(Ions_E**2 - Ions_mass**2); #% MeV/c
Ions_Pspread =0 #% percent
Ions_beta_gamma = Ions_P / Ions_mass;
print(Ions_beta_gamma,"Ions_beta_gamma")
Ions_emitt = Ions_beta_gamma *10;# mm.mrad, initial emittance
Ions_x0 = 0; #% mm, closed orbit (orbit at cooler entrance)
Ions_y0 = 0; #% mm, closed orbit (orbit at cooler entrance)
Ions_xp0 = 0; #% mrad, closed orbit (orbit at cooler entrance)
Ions_yp0 = 0;# % mrad, closed orbit (orbit at cooler entrance)

#%% Cooler params
Cooler_Vele = 9.43015738e-02 #% c
Cooler_L = 1.5;# % m, cooler length
Cooler_B = 0.06; #% T 0.07
Cooler_r0 = 24.8;# % mm, electron beam radius
#Cooler_Ne = 8.27e13;# % electron number density I=0.6A
Cooler_Ne = 3.945e+13# 6.8e+13;# % electron number density #/m^3   I= 350mA
I=0.35
Cooler_beta = 1.9; #% m, beta function in the middle of the cooler
Cooler_beta_y = 6.4; #% m, beta function in the middle of the cooler
Cooler_Tr = 0.1; #% eV
Cooler_Tl = 0.01;# % eV
Neutralisation=0 # 0/1 -- 0==OFF, 1==ON
Dx=0 #dispersion m
Cooler_Gamma=(1/(Cooler_beta*1e3))

Beta_cooler = Cooler_beta;# % m, desired beta function at the cooler
Beta_cooler_y = Cooler_beta_y;# % m, desired beta function at the cooler
L_EC = Cooler_L; #% m

In [ ]:
Nions = 5e8; #% total number of ions circulating in the ring (not important if
                #% we ignore other collective effects)

N_particles = 10000; #% number of simulated macroparticles

Lcirc = 25*np.pi; #% m, LEIR circumference length

#%% Twiss parameters
Beta = Cooler_beta; #% m, desired beta function at the cooler
Beta_y = Cooler_beta_y; #% m, desired beta function at the cooler
L_EC = Cooler_L; #% m

BunchT = RFT.Bunch6d_twiss()
BunchT.emitt_x = Ions_emitt; #% mm.mrad
BunchT.emitt_y = Ions_emitt; #% mm.mrad
BunchT.alpha_x = (L_EC/2) / Beta; #% converging beam
BunchT.alpha_y = (L_EC/2) / Beta_y; #%
BunchT.beta_x = Beta + (L_EC/2)**2 / Beta; #% m
BunchT.beta_y = Beta_y + (L_EC/2)**2 / Beta_y; #% m


#%% Create bunch
B0 = RFT.Bunch6d(Ions_mass, Nions, Ions_Q, Ions_P, BunchT, N_particles);

X_XP_Y_YP = B0.get_phase_space('%x %xp %y %yp');# % transverse phase space

#Setting starting parametters for chosen ions
#X_XP_Y_YP[1][1]=5
#X_XP_Y_YP[1][0]=4
#
# X_XP_Y_YP[0][0]=0
# X_XP_Y_YP[0][1]=0
# X_XP_Y_YP[0][2]=1
# X_XP_Y_YP[0][3]=0
#
#
# X_XP_Y_YP[1][0]=0
# X_XP_Y_YP[1][1]=0
# X_XP_Y_YP[1][2]=1
# X_XP_Y_YP[1][3]=0

#%% add Gaussian energy spread and uniform longitidinal distribution + dispersion
np.random.seed(seed=12345)
D=np.random.randn(N_particles,1)* Ions_Pspread/100

spread = 1 + D ; #% momentum spread factor
X_XP_Y_YP[:,0]=(D*Dx*1000)[:,0] +X_XP_Y_YP[:,0] #Dispersion
Vz = B0.get_phase_space('%Vz') * spread; #% c, apply the momentum spread to Vz

T = np.random.rand(N_particles, 1) * Lcirc * 1e3 / Vz; #% mm/c, uniform distribution
P = Ions_P * spread; #% MeV/c, apply the momentum spread to P

#%% final bunch
B0 = RFT.Bunch6d(Ions_mass, Nions, Ions_Q, np.hstack((X_XP_Y_YP, T, P )));



In [ ]:
import sys
sys.path.append(RFTrackPath)
import RF_Track as RFT

Nx = 20;
Ny = 20;
Nz = 2;

E = RFT.ElectronCooler(Cooler_L, Cooler_r0/1e3, Cooler_r0/1e3);
E.set_Q(-1); #% electron charge
E.set_static_Bfield(0.0, 0.0, Cooler_B);
E.set_temperature(Cooler_Tr, Cooler_Tl);

E.set_electron_mesh(Nx, Ny, Nz, Cooler_Ne, 0, 0, Cooler_Vele);

EC=E

In [ ]:
#%% LEIR params
Lcirc = 25*np.pi; #% m, LEIR circumference length
L = Lcirc - L_EC; #% m

LEIR_momentum_compaction = 0.124; #% LEIR momentum compaction
Qx = 1.82; #% 2pi, tune x
Qy = 2.72; #% 2pi, tune y
DQx = -0.5; #% 2pi, chromaticity x
DQy = -1.0; #% 2pi, chromaticity y

#% Twiss parameters at the cooler start
alpha_x = (L_EC/2) / Beta_cooler; #% converging beam
alpha_y = (L_EC/2) / Beta_cooler_y; #%
beta_x = Beta_cooler + (L_EC/2)**2 / Beta_cooler; #% m
beta_y = Beta_cooler_y + (L_EC/2)**2 / Beta_cooler_y; #% m

#% Crate one matching transfer matrix for the rest of the ring
TWISS = np.zeros((2, 11))
TWISS[0,:] = [ 0, beta_x, -alpha_x, 0,  beta_y, -alpha_y, 0 , Dx, 0, 0, 0]; #% cooler end
TWISS[1,:] = [ L, beta_x,  alpha_x, Qx, beta_y,  alpha_y, Qy, Dx, 0, 0, 0]; #% cooler start

RING = RFT.TransferLine(TWISS, DQx, DQy, LEIR_momentum_compaction, Ions_P);

#%% Create the lattice
L = RFT.Lattice()
L.append(EC)
L.append(RING)

In [ ]:
transfer_line=L
beam=B0

# Create empty lists to store RF track data
x_rf_list = []
px_rf_list = []
hist_1_rf_list = []
hist_2_rf_list = []
width_rf_list = []

num_turns = int(1e5)
cooler_boolean = True
data_num_turns=int(1e5)
save_interval = 100

lower_percentile = 2.5
upper_percentile = 97.5

# Specify the file paths where you want to save the data
file_path_no_cooler_rf = 'results/rf_no_cooler.npz'
file_path_cooler_rf = 'results/rf_cooler.npz'

from tqdm import tqdm

for i in tqdm(range(num_turns)):
    transfer_line.track(beam)

    if cooler_boolean:
        E.track(beam)

    if i % save_interval == 0:
        x_rf = beam.get_phase_space("%x") * 1e-3
        px_rf = beam.get_phase_space("%xp") * 1e-3
        x_rf_list.append(x_rf)
        px_rf_list.append(px_rf)
        hist_1_rf_list.append(np.percentile(x_rf, lower_percentile))
        hist_2_rf_list.append(np.percentile(x_rf, upper_percentile))
        width_rf_list.append(np.percentile(x_rf, upper_percentile) - np.percentile(x_rf, lower_percentile))

if num_turns >= data_num_turns:
    # Save the lists as a single compressed archive file
    if not cooler_boolean:
        np.savez(file_path_no_cooler_rf, x_rf=x_rf_list, px_rf=px_rf_list, hist_1_rf=hist_1_rf_list, hist_2_rf=hist_2_rf_list, width_rf=width_rf_list)
    if cooler_boolean:
        np.savez(file_path_cooler_rf, x_rf=x_rf_list, px_rf=px_rf_list, hist_1_rf=hist_1_rf_list, hist_2_rf=hist_2_rf_list, width_rf=width_rf_list)    